# PyData NLP workshop

In [28]:
# setup
import sys
import subprocess
import pkg_resources
from collections import Counter
import re


required = {'spacy', 'scikit-learn', 'spacy-transformers'}
installed = {pkg.key for pkg in pkg_resources.working_set}
missing = required - installed

if missing:
    python = sys.executable
    subprocess.check_call([python, '-m', 'pip', 'install', *missing], stdout=subprocess.DEVNULL)

import spacy
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation


## Tokenization
This section shows some of the considerations to make when tokenizing your data.

Token = "Useful semantic unit"

But what does that mean? This section will detail some considerations here.

In [29]:
# importing different languages in spacy
# blank English model
from spacy.lang.en import English
en = English()
# blank Chinese model
# to run, will need to install jieba tokenizer (optional)
#!pip install jieba
from spacy.lang.zh import Chinese

zh = spacy.lang.zh.Chinese()
zh_text = '我们正在做NLP。'
print('Tokenize in Chinese:', [x.text for x in zh(zh_text)])
print('Tokenize in English:', [x.text for x in en(zh_text)])

Tokenize in Chinese: ['我们', '正在', '做', 'NLP', '。']
Tokenize in English: ['我们正在做NLP', '。']


In [30]:
# lowercasing
text = 'We are doing NLP.'
print('Base python: ', text.lower())
print('SpaCy:', [x.lower_ for x in en(text)])

Base python:  we are doing nlp.
SpaCy: ['we', 'are', 'doing', 'nlp', '.']


In [31]:
# handling non-alpha
text = 'We are doing NLP.'
# base python
strip_punct = '[^A-Za-z0-9 ]'
print(re.sub(strip_punct, '', text))
# spacy
print([x.text for x in en(text) if x.is_alpha])

We are doing NLP
['We', 'are', 'doing', 'NLP']


In [32]:
# but what about contractions?
text = "We're doing NLP."
# base python
strip_punct = '[^A-Za-z0-9 ]'
print('Just removing punctuation:', re.sub(strip_punct, '', text))
# spacy
print('Removing non-alpha', [x.text for x in en(text) if x.is_alpha])

Just removing punctuation: Were doing NLP
Removing non-alpha ['We', 'doing', 'NLP']


You can see here that the is_alpha flag is False for any tokens that have non-alpha characters.  We'll look into a better way for dealing with contractions later.

### Exercise: Create a tokenizer
In this exercise, you will make a function that uses spaCy's base English model to tokenize a dataset according to specific parameters.  The functions will take a list of documents and output a list of tokens.  In this case we're interested in outputting strings, rather than spaCy tokens.

In [33]:
# data
text_data = ["I'm taking a course at Harvard.",
            "I'm learning about Natural Language Processing.",
            "We are studying tokenization, vectorization and modelling.",
            "Check out the course on Github: https://github.com/bpben/nlp_lessons"]

In [34]:
# initialize model
en = English()

def tokenize_base(docs, model=en):
    # tokenizer that just parses using spaCy's base model
    tokenized_docs = []
    for d in docs:
        parsed = model(d)
        tokenized_docs.append([t.text for t in parsed])
    return(tokenized_docs)

def tokenize_lower_alpha(docs, model=en):
    # tokenizer that lowercases and removes any non-alpha character
    tokenized_docs = []
    for d in docs:
        parsed = model(d)
        tokenized_docs.append([t.lower_ for t in parsed if t.is_alpha])
    return(tokenized_docs)

def tokenize_lower_alpha_url(docs, model=en):
    # tokenizer that lowercases, removes any non-alpha character and removes urls
    tokenized_docs = []
    for d in docs:
        parsed = model(d)
        tokenized_docs.append([t.lower_ for t in parsed if (t.is_alpha)&(not t.like_url)])
    return(tokenized_docs)

### Lemmatization and Stemming
Though word tense can sometimes carry with it a lot of useful information, a lot of time it might be useful to reduce words to their common root.  For example, the word "be" has various forms like "are", "is", "been".  We might not want our vocabulary to contain all these forms and rather count them all as instances of "be".

In [35]:
# read in English model with tagging/entity pipeline components
# you will need to run the line below beforehand
#!python -m spacy download en_core_web_sm
nlp = spacy.load('en_core_web_sm')
text = 'I am taking an NLP course.'
print(text)
print([x.lemma_ for x in nlp(text)])

I am taking an NLP course.
['-PRON-', 'be', 'take', 'an', 'NLP', 'course', '.']


### Stop words
Dealing with stop words involves making some pretty impactful decisions with your data.  Refer to the slides for details.  Here, we just remove stop words based on [spaCy's default set](https://github.com/explosion/spaCy/blob/master/spacy/lang/en/stop_words.py).

In [36]:
en = English()
text = 'In June 2020, I took a course at Harvard Extension School in Cambridge.'
print(text)
print([x.text for x in en(text) if not x.is_stop])

In June 2020, I took a course at Harvard Extension School in Cambridge.
['June', '2020', ',', 'took', 'course', 'Harvard', 'Extension', 'School', 'Cambridge', '.']


### Non-standard tokens (e.g. named-entities)
In text, some some n-grams should not be treated as a concatenation of unigrams.  For example, New York City is fundamentally different from the individual words "new", "york" and "city".

Here we attempt to deal with some of these non-standard tokens

In [37]:
# urls
# base python
# regex from textacy: https://github.com/chartbeat-labs/textacy
SHORT_URL_REGEX = re.compile(
    r"(?:^|(?<![\w/.]))"
    # optional scheme
    r"(?:(?:https?://)?)"
    # domain
    r"(?:\w-?)*?\w+(?:\.[a-z]{2,12}){1,3}"
    r"/+",
    flags=re.IGNORECASE)
text = 'Check out these courses: https://www.summer.harvard.edu/'
print(text)
print(SHORT_URL_REGEX.sub('', text))
# spacy
print([x for x in en(text) if not x.like_url])
# spacy - replace with a standard token
print(['-URL-' if x.like_url else x for x in en(text)])

Check out these courses: https://www.summer.harvard.edu/
Check out these courses: 
[Check, out, these, courses, :]
[Check, out, these, courses, :, '-URL-']


In [38]:
# named-entities
# read in English model with tagging/entity pipeline components
nlp = spacy.load('en_core_web_sm')
text = 'I am taking an NLP course at Harvard starting July 19th, 2020'
parsed = nlp(text)
# look at the individual tokens
tokens = [t for t in parsed]
print(tokens)
# look at the identified named-entities and their types
for e in parsed.ents:
    print(e, type(e), e.label_, spacy.explain(e.label_))

[I, am, taking, an, NLP, course, at, Harvard, starting, July, 19th, ,, 2020]
NLP <class 'spacy.tokens.span.Span'> ORG Companies, agencies, institutions, etc.
Harvard <class 'spacy.tokens.span.Span'> ORG Companies, agencies, institutions, etc.
July 19th, 2020 <class 'spacy.tokens.span.Span'> DATE Absolute or relative dates or periods


### Exercise: A comprehensive tokenization pipeline

In [39]:
# data
text_data = ["I'm taking a course at Harvard.",
            "I'm learning about Natural Language Processing.",
            "We are studying tokenization, vectorization and modelling.",
            "Check out the course on Github: https://github.com/bpben/nlp_lessons"]

In [40]:
# initialize model
nlp = spacy.load('en_core_web_sm')

def tokenize_full(docs, model=nlp, 
                  entities=False, 
                  stop_words=False, 
                  lowercase=True, 
                  alpha_only=True, 
                  lemma=True):
    """Full tokenizer with flags for processing steps
    entities: If False, replaces with entity type
    stop_words: If False, removes stop words
    lowercase: If True, lowercases all tokens
    alpha_only: If True, removes all non-alpha characters
    lemma: If True, lemmatizes words
    """
    tokenized_docs = []
    for d in docs:
        parsed = model(d)
        # token collector
        tokens = []
        # index pointer
        i = 0
        # entity collector
        ent = ''
        for t in parsed:
            # only need this if we're replacing entities
            if not entities:
                # replace URLs
                if t.like_url:
                    tokens.append('URL')
                    continue
                # if there's entities collected and current token is non-entity
                if (t.ent_iob_=='O')&(ent!=''):
                    tokens.append(ent)
                    ent = ''
                    continue
                elif t.ent_iob_!='O':
                    ent = t.ent_type_
                    continue
            # only include stop words if stop words==True
            if (t.is_stop)&(not stop_words):
                continue
            # only include non-alpha is alpha_only==False
            if (not t.is_alpha)&(alpha_only):
                continue
            if lemma:
                t = t.lemma_
            else:
                t = t.text
            if lowercase:
                t.lower()
            tokens.append(t)
        tokenized_docs.append(tokens)
    return(tokenized_docs)

In [41]:
tokenize_full(text_data, stop_words=True, alpha_only=False, entities=True)

[['-PRON-', 'be', 'take', 'a', 'course', 'at', 'Harvard', '.'],
 ['-PRON-', 'be', 'learn', 'about', 'Natural', 'Language', 'Processing', '.'],
 ['-PRON-',
  'be',
  'study',
  'tokenization',
  ',',
  'vectorization',
  'and',
  'modelling',
  '.'],
 ['check',
  'out',
  'the',
  'course',
  'on',
  'Github',
  ':',
  'https://github.com/bpben/nlp_lessons']]

## Word counts
A very basic way to use a sanitized list of tokens is to do a word count.  This unlocks a lot of insights right off and is an important step in exploratory data analysis in text.

In [42]:
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from spacy.lang.en import English
en = English()

def simple_tokenizer(doc, model=en):
    # a simple tokenizer for individual documents (different from above)
    tokenized_docs = []
    parsed = model(doc)
    return([t.lower_ for t in parsed if (t.is_alpha)&(not t.like_url)])

In [43]:
# data
text_data = ["I'm taking a course at Harvard.",
            "I'm learning about Natural Language Processing.",
            "We are studying tokenization, vectorization and modelling.",
            "Check out the course on Github: https://github.com/bpben/nlp_lessons"]
tokenized = [simple_tokenizer(doc) for doc in text_data]

In [44]:
# base python: create an make use of a Counter object
counts = [Counter(d) for d in tokenized]
print('List of counts:', counts)
# sum together all counts
all_counts = Counter()
for d in tokenized:
    all_counts += Counter(d)
print(counts)
print('\nCombined count:', all_counts)

List of counts: [Counter({'i': 1, 'taking': 1, 'a': 1, 'course': 1, 'at': 1, 'harvard': 1}), Counter({'i': 1, 'learning': 1, 'about': 1, 'natural': 1, 'language': 1, 'processing': 1}), Counter({'we': 1, 'are': 1, 'studying': 1, 'tokenization': 1, 'vectorization': 1, 'and': 1, 'modelling': 1}), Counter({'check': 1, 'out': 1, 'the': 1, 'course': 1, 'on': 1, 'github': 1})]
[Counter({'i': 1, 'taking': 1, 'a': 1, 'course': 1, 'at': 1, 'harvard': 1}), Counter({'i': 1, 'learning': 1, 'about': 1, 'natural': 1, 'language': 1, 'processing': 1}), Counter({'we': 1, 'are': 1, 'studying': 1, 'tokenization': 1, 'vectorization': 1, 'and': 1, 'modelling': 1}), Counter({'check': 1, 'out': 1, 'the': 1, 'course': 1, 'on': 1, 'github': 1})]

Combined count: Counter({'i': 2, 'course': 2, 'taking': 1, 'a': 1, 'at': 1, 'harvard': 1, 'learning': 1, 'about': 1, 'natural': 1, 'language': 1, 'processing': 1, 'we': 1, 'are': 1, 'studying': 1, 'tokenization': 1, 'vectorization': 1, 'and': 1, 'modelling': 1, 'check'

In [45]:
# scikit-learn's countvectorizer
# use our custom tokenizer
cv = CountVectorizer(tokenizer=simple_tokenizer)
# outputs sparse array, want to use a normal numpy array
v = cv.fit_transform(text_data).toarray()
# get_feature_names gets the vocabulary of the vectorizer in order
dict(zip(cv.get_feature_names(), v.sum(axis=0)))
# result is the same as above

{'a': 1,
 'about': 1,
 'and': 1,
 'are': 1,
 'at': 1,
 'check': 1,
 'course': 2,
 'github': 1,
 'harvard': 1,
 'i': 2,
 'language': 1,
 'learning': 1,
 'modelling': 1,
 'natural': 1,
 'on': 1,
 'out': 1,
 'processing': 1,
 'studying': 1,
 'taking': 1,
 'the': 1,
 'tokenization': 1,
 'vectorization': 1,
 'we': 1}

### Exercise: Sentiment analysis with word counts
Imagine you are a hot dog restaurant owner and you want to analyze a corpus of reviews from diners to see whether people generally think your hot dogs are "good" or "bad".  Specifically, you're going to count up the number of times the word "good" and word "bad" appears.  Depending on how you process the text, you will arrive at different conclusions.  Try a couple ways to see what I mean.

You might also want to think about whether all the reviews are relevant.  Those sorts of choices may also affect your results.  Is there an automatic way you can remove non-relevant reviews?

In [46]:
reviews = ['These hot dogs are really good.',
          'These hot dogs are really bad.',
          'Good hot dogs!',
          'The hot dogs pair well with a Good Humor bar.',
          "I didn't eat anything, I felt bad."]

## Word counts with scikit-learn

In [47]:
# scikit-learn's countvectorizer
# use our custom tokenizer
cv = CountVectorizer(tokenizer=simple_tokenizer)

In [48]:
# data
text_data = ["I'm taking a course at Harvard.",
            "I'm learning about Natural Language Processing.",
            "We are studying tokenization, vectorization and modelling.",
            "Check out the course on Github: https://github.com/bpben/nlp_lessons"]
# outputs sparse array, want to use a normal numpy array
v = cv.fit_transform(text_data).toarray()
# get_feature_names gets the vocabulary of the vectorizer in order
dict(zip(cv.get_feature_names(), v.sum(axis=0)))

{'a': 1,
 'about': 1,
 'and': 1,
 'are': 1,
 'at': 1,
 'check': 1,
 'course': 2,
 'github': 1,
 'harvard': 1,
 'i': 2,
 'language': 1,
 'learning': 1,
 'modelling': 1,
 'natural': 1,
 'on': 1,
 'out': 1,
 'processing': 1,
 'studying': 1,
 'taking': 1,
 'the': 1,
 'tokenization': 1,
 'vectorization': 1,
 'we': 1}

Works as expected.  Why don't we try this on Assignment 1's dataset?

In [53]:
# you will need to change this to where ever the file is stored
data_location = '../data/movie_reviews_subset.pkl'
with open(data_location, 'rb') as f:
    all_text = pickle.load(f)
# corpora size
print([(k, len(all_text[k])) for k in all_text])
# for simplicity, let's split these into separate sets
neg, pos = all_text.values()

[('neg', 1233), ('pos', 1266)]


In [54]:
# running this on negative reviews
cv = CountVectorizer(tokenizer=simple_tokenizer)
neg_vectors = cv.fit_transform(neg).toarray()
# get_feature_names gets the vocabulary of the vectorizer in order
word_count = dict(zip(cv.get_feature_names(), neg_vectors.sum(axis=0)))
# get the top 10 words
sorted(word_count.items(), key=lambda x: x[1], reverse=True)[:10]

[('the', 15365),
 ('a', 7548),
 ('and', 6978),
 ('to', 6780),
 ('of', 6402),
 ('is', 4952),
 ('it', 4354),
 ('i', 4248),
 ('in', 4203),
 ('this', 3837)]

In [55]:
# now do it for positive reviews
cv = CountVectorizer(tokenizer=simple_tokenizer)
pos_vectors = cv.fit_transform(pos).toarray()
# get_feature_names gets the vocabulary of the vectorizer in order
word_count = dict(zip(cv.get_feature_names(), pos_vectors.sum(axis=0)))
# get the top 10 words
sorted(word_count.items(), key=lambda x: x[1], reverse=True)[:10]

[('the', 17182),
 ('and', 8905),
 ('a', 8133),
 ('of', 7942),
 ('to', 6658),
 ('is', 5793),
 ('in', 5093),
 ('it', 4553),
 ('i', 3943),
 ('that', 3521)]

These words aren't particularly informative about the content.  Sklearn's CountVectorizer has some additional options that may lead to somewhat more informative frequent terms.

In [56]:
for corpus in [neg, pos]:
    cv = CountVectorizer(tokenizer=simple_tokenizer, min_df=0.01, max_df=0.9,
                        stop_words='english')
    vectors = cv.fit_transform(corpus).toarray()
    # get_feature_names gets the vocabulary of the vectorizer in order
    word_count = dict(zip(cv.get_feature_names(), vectors.sum(axis=0)))
    # get the top 10 words
    print(sorted(word_count.items(), key=lambda x: x[1], reverse=True)[:10])

/opt/anaconda3/envs/mainpy3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ca', 'nt'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


[('movie', 2297), ('film', 1797), ('like', 1085), ('just', 984), ('bad', 668), ('did', 637), ('good', 632), ('really', 620), ('time', 560), ('does', 521)]
[('film', 2122), ('movie', 1904), ('like', 891), ('good', 811), ('just', 713), ('great', 654), ('story', 625), ('time', 604), ('really', 539), ('does', 508)]


This is better, but it seems like we'd have to tweak these thresholds a lot and carefully choose our stop words.  Is there a more standard way to extract the most informative words from documents?

## Term Frequency-Inverse Document Frequency (TF-IDF)
See the slides for more information on this.  In this section we'll show how TF-IDF is essentially just a weighting of the count vectors.  We'll then use sklearn's built-in TfidfVectorizer on our sentiment corpora.

In [57]:
docs = ['The movie was good',
        'The movie was bad',
        'The movie was great']

cv = CountVectorizer(tokenizer=simple_tokenizer)
vecs = cv.fit_transform(docs).toarray()
# we'll use pandas DF for easier display
pd.DataFrame(vecs, columns=cv.get_feature_names())

bad  good  great  movie  the  was
0    0     1      0      1    1    1
1    1     0      0      1    1    1
2    0     0      1      1    1    1

You'll notice that `vecs` contains the term frequencies.  If we use sklearn's `TfidfVectorizer`, it will calculate those term counts and then multiply them by the Inverse Document Frequency (IDF).

The formula sklearn uses is a bit different from the textbook:

$$log(\frac{N+1}{df(t)+1}) + 1$$

Where $N$ is the number of documents.  It also normalizes this value to account for different size vectors (see slides).

In [58]:
tfidf = TfidfVectorizer(tokenizer=simple_tokenizer)
# we'll use pandas DF for easier display
tfidf_vecs = tfidf.fit_transform(docs).toarray()
tfidf_df = pd.DataFrame(tfidf_vecs, columns=tfidf.get_feature_names())
tfidf_df

bad     good    great     movie       the       was
0  0.00000  0.69903  0.00000  0.412859  0.412859  0.412859
1  0.69903  0.00000  0.00000  0.412859  0.412859  0.412859
2  0.00000  0.00000  0.69903  0.412859  0.412859  0.412859

You can see that the discriminative words (i.e. bad, good, great) have higher weight than the non-discriminative words.  

We see this at the document level, but is there a way we could get some kind of aggregate measure of discriminative words?

### Exercise: Find the top 3 discriminative words
Use the dataset above to try and identify the words that, across the corpus, are particularly representative of content.

Hint: Think about what a weight of zero versus weight of non-zero means.

In [59]:
def top_tfidf_words(tfidf_df):
    return(tfidf_df[tfidf_df>0].mean(axis=0))

In [60]:
top_tfidf_words(tfidf_df)

bad      0.699030
good     0.699030
great    0.699030
movie    0.412859
the      0.412859
was      0.412859
dtype: float64

Now let's run that on our movie reviews dataset.

In [61]:
for corpus in [neg, pos]:
    # adding in a minimum document frequency, so words need to occur at least somewhat often
    tfidf = TfidfVectorizer(tokenizer=simple_tokenizer, min_df=0.02)
    vectors = tfidf.fit_transform(corpus).toarray()
    tfidf_df = pd.DataFrame(vectors, columns=tfidf.get_feature_names())
    # get representative words
    tfidf_word_count = top_tfidf_words(tfidf_df)
    # get the top 10 words
    print(tfidf_word_count.sort_values().iloc[-10:])

children    0.151333
monster     0.152135
dr          0.153720
cute        0.156171
island      0.157478
space       0.165138
theater     0.165205
nt          0.167994
game        0.189159
the         0.267655
dtype: float64
and         0.153424
oscar       0.154922
dance       0.155299
edge        0.159386
rock        0.162227
western     0.165590
ryan        0.167666
japanese    0.172789
dr          0.177205
the         0.274663
dtype: float64


These are somewhat useful aggregate measures.  But most of the information in TF-IDF is document-specific.


## Topic models: Non-negative Matrix Factorization and Latent Dirichlet Allocation

In [62]:
def display_components(model, word_features, top_display=5):
    # utility for displaying respresentative words per component for topic models
    for topic_idx, topic in enumerate(model.components_):
        print("Topic %d:" % (topic_idx))
        top_words_idx = topic.argsort()[::-1][:top_display]
        top_words = [word_features[i] for i in top_words_idx]
        print(" ".join(top_words))

In [64]:
# in this case, excluding standard english stop words
# creating a combined review dataset
all_reviews = neg+pos
tfidf = TfidfVectorizer(tokenizer=simple_tokenizer, stop_words='english')
tfidf_vecs = tfidf.fit_transform(all_reviews)
cv = CountVectorizer(tokenizer=simple_tokenizer, stop_words='english')
count_vecs = cv.fit_transform(all_reviews)

/opt/anaconda3/envs/mainpy3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ca', 'nt'] not in stop_words.
  'stop_words.' % sorted(inconsistent))
/opt/anaconda3/envs/mainpy3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ca', 'nt'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [65]:
# choose the number of components (topics)
n_components = 10
# basic configuration
nmf = NMF(n_components=n_components)
# NMF requires tfidf, not word counts
# same syntax as vectorizer
nmf_vecs = nmf.fit_transform(tfidf_vecs)
# LDA uses word counts
lda = LatentDirichletAllocation(n_components=n_components)
lda_vecs = lda.fit_transform(count_vecs)

Both NMF and LDA provide a components matrix which corresponds to the loading of each word on each topic.  Higher values means the word is more relevant to that topic.

In [66]:
print(nmf.components_)

[[1.14459699e-03 0.00000000e+00 3.70323998e-03 ... 1.90700493e-05
  6.86192149e-04 6.86192149e-04]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 5.29703942e-05
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 3.42693943e-04
  2.33893836e-03 2.33893836e-03]
 ...
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 2.13925918e-03 ... 6.81188262e-05
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 4.35862023e-05 0.00000000e+00 ... 1.53815885e-04
  0.00000000e+00 0.00000000e+00]]


For evaluating performance, both methods use different ways to quantify the loss from using the topic model versus the actual data.  (In the matrix formulation, $UV$ rather than $X$).  For NMF, it's reconstruction error, which is more directly the difference between the matrix decomposition and the actual data.  For LDA, it uses [ELBO](https://en.wikipedia.org/wiki/Evidence_lower_bound), which is a too complicated to explain here.  In both, higher values means worse performance.  They can't be compared to one another, though.

In [67]:
print(nmf.reconstruction_err_, lda.bound_)

48.814189927095335 7011.152878238849


In [68]:
display_components(nmf, tfidf.get_feature_names())

Topic 0:
film films good seen does
Topic 1:
movie watch good movies time
Topic 2:
man life story family young
Topic 3:
series episode episodes tv season
Topic 4:
br money music audience thing
Topic 5:
great best good action love
Topic 6:
did like just people really
Topic 7:
book read story novel character
Topic 8:
bad acting good worst movies
Topic 9:
horror effects special budget gore


In [69]:
display_components(lda, cv.get_feature_names())

Topic 0:
movie film like great good
Topic 1:
film movie story like good
Topic 2:
film movie good just like
Topic 3:
film movie like just make
Topic 4:
movie film like just good
Topic 5:
film like way just characters
Topic 6:
like just time does movie
Topic 7:
movie film like just people
Topic 8:
movie like film just good
Topic 9:
movie film like story good


NMF seems to have come up with some reasonable topics, but LDA doesn't seem to work particularly well here.  It may make sense to try some additional token processing and see how that affects what we get out of the topic modelling process.

### Exercise: Tokenization decisions and topic models

In [70]:
# initialize model
nlp = spacy.load('en_core_web_sm')

def tokenize_full(docs, model=nlp, 
                  entities=False, 
                  stop_words=False, 
                  lowercase=True, 
                  alpha_only=True, 
                  lemma=True):
    """Full tokenizer with flags for processing steps
    entities: If False, replaces with entity type
    stop_words: If False, removes stop words
    lowercase: If True, lowercases all tokens
    alpha_only: If True, removes all non-alpha characters
    lemma: If True, lemmatizes words
    """
    tokenized_docs = []
    for d in docs:
        parsed = model(d)
        # token collector
        tokens = []
        # index pointer
        i = 0
        # entity collector
        ent = ''
        for t in parsed:
            # only need this if we're replacing entities
            if not entities:
                # replace URLs
                if t.like_url:
                    tokens.append('URL')
                    continue
                # if there's entities collected and current token is non-entity
                if (t.ent_iob_=='O')&(ent!=''):
                    tokens.append(ent)
                    ent = ''
                    continue
                elif t.ent_iob_!='O':
                    ent = t.ent_type_
                    continue
            # only include stop words if stop words==True
            if (t.is_stop)&(not stop_words):
                continue
            # only include non-alpha is alpha_only==False
            if (not t.is_alpha)&(alpha_only):
                continue
            if lemma:
                t = t.lemma_
            else:
                t = t.text
            if lowercase:
                t.lower()
            tokens.append(t)
        tokenized_docs.append(tokens)
    return(tokenized_docs)

In [71]:
tokenized = tokenize_full(all_reviews, entities=True)

In [72]:
# if passing a list of tokens to a vectorizer, you can use the following syntax
tfidf = TfidfVectorizer(tokenizer=lambda doc: doc, lowercase=False)
tfidf_vecs = tfidf.fit_transform(tokenized)
cv = CountVectorizer(tokenizer=lambda doc: doc, lowercase=False)
count_vecs = cv.fit_transform(tokenized)

In [73]:
n_components = 10
nmf = NMF(n_components=n_components)
nmf_vecs = nmf.fit_transform(tfidf_vecs)
lda = LatentDirichletAllocation(n_components=n_components)
lda_vecs = lda.fit_transform(count_vecs)

In [74]:
display_components(nmf, tfidf.get_feature_names())

Topic 0:
film see watch good time
Topic 1:
movie watch see think time
Topic 2:
love life story family young
Topic 3:
like scene go look people
Topic 4:
series episode watch season funny
Topic 5:
br music money play audience
Topic 6:
great good actor cast acting
Topic 7:
book read story character novel
Topic 8:
bad acting see terrible waste
Topic 9:
game video play level like


In [75]:
display_components(lda, cv.get_feature_names())

Topic 0:
Barney France take solve find
Topic 1:
film movie good br scene
Topic 2:
movie like film time scene
Topic 3:
movie like bad good film
Topic 4:
movie film good like time
Topic 5:
movie film like good character
Topic 6:
film br character scene come
Topic 7:
film movie good like story
Topic 8:
film like time scene movie
Topic 9:
movie film watch good like
